In [1]:
import pandas as pd
from datetime import date

import datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from prophet import Prophet

Загрузка тренировочного датасета. В процессе разработки все подготовительные работы с датасетом сделаны в другом ноутбуке, здесь мы используем подготовленный датасет

In [2]:
pr_df = pd.read_csv('D:\data_science\demand_forecast_for_products\sp_sales_task\pr_df.csv')
sales_df = pd.read_csv('D:\data_science\demand_forecast_for_products\sp_sales_task\sales_df_train.csv')
sales_submission = pd.read_csv('D:\data_science\demand_forecast_for_products\sp_sales_task\sales_submission.csv')
st_df = pd.read_csv('D:\data_science\demand_forecast_for_products\sp_sales_task\st_df.csv')

In [3]:
# Соединяем все таблицы
df_1 = pd.merge(sales_df, st_df, how='left', left_on='st_id', right_on='st_id')
df = pd.merge(df_1, pr_df, how='left', left_on='pr_sku_id', right_on='pr_sku_id')

In [4]:
# Преобразуем хэш-признаки на основе частотности 
df['fe_st_id'] = df['st_id'].map(df['st_id'].value_counts(normalize=False))
df['fe_pr_sku_id'] = df['pr_sku_id'].map(df['pr_sku_id'].value_counts(normalize=False))
df['fe_st_city_id'] = df['st_city_id'].map(df['st_city_id'].value_counts(normalize=False))
df['fe_st_division_code'] = df['st_division_code'].map(df['st_division_code'].value_counts(normalize=False))
df['fe_pr_group_id'] = df['pr_group_id'].map(df['pr_group_id'].value_counts(normalize=False))
df['fe_pr_cat_id'] = df['pr_cat_id'].map(df['pr_cat_id'].value_counts(normalize=False))
df['fe_pr_subcat_id'] = df['pr_subcat_id'].map(df['pr_subcat_id'].value_counts(normalize=False))

In [5]:
df = df.rename(columns = {'pr_sales_in_rub' : 'y'})
df['ds'] = pd.to_datetime(df['date'])

In [6]:
df = df.drop(['st_id', 'pr_sku_id', 'st_city_id',
              'st_division_code', 'pr_group_id',
                  'pr_cat_id', 'pr_subcat_id',
                  'pr_promo_sales_in_units',
                   'pr_promo_sales_in_rub', 'date', 'pr_sales_in_units'], axis=1)

Разделяем данные по времени на тренировочные и тестовые. 

In [7]:
date_lag = 15
predictions_period = df['ds'].max() - timedelta(date_lag)
train = df.loc[df['ds'] < predictions_period]
test = df.loc[df['ds'] >= predictions_period]

In [8]:
# Проверяем, что разделение прошло успешно
print("Даты тренировочного датасета: с", train['ds'].min(), "по:", train['ds'].max())
print("Даты тренировочного датасета: с", test['ds'].min(), "по:", test['ds'].max())

Даты тренировочного датасета: с 2022-08-01 00:00:00 по: 2023-07-02 00:00:00
Даты тренировочного датасета: с 2023-07-03 00:00:00 по: 2023-07-18 00:00:00


In [9]:
# Создим признаки и целевые признаки.
X_train = train.drop(['y', 'ds'], axis=1)
y_train = train['y']

In [10]:
# Перебор очень долгий. В связи с тем, что мы пока тестируем модели, используем дефолтные параметры.

# %%time

# # Попробуем изменить гиперпараметры. Будем использовать кросс-валидацию с использованием 
# # библиотеки TimeSeriesSplit, что исключит подглядывание 
# model_cbr_tune = CatBoostRegressor(loss_function='RMSE')
# tscv = TimeSeriesSplit(n_splits=5)
# parameters_cbr_tune = {'iterations': [1000],
#         'learning_rate': [0.03],#, 0.1],
#         'depth': [6],#, 8],
#         'l2_leaf_reg': [1]#, 3]
#               }
# model_cbr_tune = GridSearchCV(model_cbr_tune, 
#                         parameters_cbr_tune, 
#                         cv = tscv, 
#                         n_jobs=-1,
#                               scoring = 'neg_mean_squared_error',
#                              verbose=True).fit(X_train, y_train, 
#                                                verbose=False, plot=True)


Для корректного создания резолютирующего файла предсказания делаем по всем комбинациям магазин-товар. 

In [11]:
model_cbr_tune = CatBoostRegressor(loss_function='RMSE').fit(X_train, y_train, 
                                               verbose=False, plot=True)


model_cbr_tune.get_feature_importance(prettified=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

,Feature Id,Importances
0,fe_pr_sku_id,27.990605
1,fe_pr_cat_id,21.522737
2,fe_pr_subcat_id,17.985495
3,fe_pr_group_id,15.610606
4,pr_sales_type_id,5.099075
5,st_type_loc_id,3.583805
6,fe_st_id,3.377191
7,fe_st_city_id,1.778352
8,fe_st_division_code,1.354877
9,st_type_size_id,1.091654


In [12]:
# Определяем параметры перебора по связкам магазин-товар.
a = set(test['fe_st_id'])
b = set(test['fe_pr_sku_id'])

In [13]:
%%time
sales_submission_test = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date']) #Для вычисления метрики
for i in a:
    for j in b:
        test_pred = test[(test['fe_st_id'] == i) & (test['fe_pr_sku_id'] == j)] # Выделяем часть датасета в разрезе нужной связки
       
        df_1 = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date', 'y']) # Резолютирующий датасет по предсказаниям

        df_1['st_id'], df_1['pr_sku_id'], df_1['date'], df_1['y'] = \
            test_pred['fe_st_id'], test_pred['fe_pr_sku_id'], test_pred['ds'], test_pred['y']
       
        X_test = test_pred.drop(['y', 'ds'], axis=1) # Выделяем данные для предсказаний

        df_1['target'] = model_cbr_tune.predict(X_test) # Предсказание

        sales_submission_test = pd.concat([sales_submission_test, df_1], ignore_index=False)

<timed exec>:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


CPU times: total: 4.67 s
Wall time: 23.5 s


In [14]:
# sales_submission_test.groupby('date').count()

In [15]:
# Вычисляем метрику
wape = 100 * (sales_submission_test['y'] - sales_submission_test['target']).abs().sum() / sales_submission_test['y'].sum()
wape

62.27562471770342

In [16]:
# Посмотрим, что получилось
sales_submission_test.reset_index(drop= True , inplace= True )
sales_submission_test

,st_id,pr_sku_id,date,y,target
0,157542,2049,2023-07-06,3094.0,2141.442126
1,157542,2049,2023-07-08,394.0,4137.182149
2,157542,2049,2023-07-05,3311.0,2141.442126
3,157542,2049,2023-07-05,826.0,4137.182149
4,157542,2049,2023-07-13,1047.0,4137.182149
...,...,...,...,...,...
41776,174687,2047,2023-07-06,231.0,230.670370
41777,174687,2047,2023-07-15,360.0,230.670370
41778,174687,2047,2023-07-03,31.0,68.178567
41779,174687,2047,2023-07-17,62.0,68.178567


In [17]:
# Создаем нужный файл в соответствии с заданием и записываем его

sales_submission_row = sales_submission_test.drop(['y'], axis=1)
sales_submission_row.to_csv('sales_submission.csv')